<a href="https://colab.research.google.com/github/mendesdyl/mendesdyl/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating an ensemble of states to show differences in the clustering 

## Import Packages and Mount Google Drive

### Importing the packages here and installing pyemma because it was not working on DT2 (DeepThought2(HPC))

In [1]:
import numpy as np
import matplotlib.pyplot as plt
#!pip install pyemma
#import pyemma
!pip install mdtraj
import mdtraj as md
import pprint
from six.moves import xrange
import re
import os
import shutil
import argparse
import time
import gzip

The whole reason I did the ensemble of states on this platform was that I would be able to use pyemme but the way Min-Kang had it implemented I did not need to use pyemma. Therefore, pyemma is commented out, MDTraj was used instead for all the calculations. 

### Import Google Drive to access the files uploaded from DeepThought2

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
# /content/gdrive/MyDrive/ensemble/
# This is the path once the drive is mounted

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Run the ensemble of states (more defenition below) 

### Convert the files from dcd and pdb to h5 format in order to process them as one large clustering

In [6]:
dyn = [150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 92, 162, 149, 161, 131, 137, 157, 158]
s=0
o=1
for i in range(0, 19):
    pdbFile = i + 1
    for d in range(1,int(dyn[i]+1)):
        cmd = "mdconvert -o /content/gdrive/MyDrive/ensemble/comb/{0}.h5 -t /content/gdrive/MyDrive/ensemble/dcd_a_ca/a_ca{1}.pdb /content/gdrive/MyDrive/ensemble/dcd_a_ca/a_ca_dyn{2}.dcd".format(s,pdbFile,o)
        os.system(cmd)
        s = s + 1
        o = o + 1

#this will create new .h5 files that include topology because that caused a ton of errors 

### Execute the top 50 least RMSD frames into new dcd and pdb files 

In [7]:
f=open('/content/gdrive/MyDrive/ensemble/cluster_id_s9.dat','r')
##Edit for your simulation
inds = []
y_m_t =[]
lines=f.readlines()
#lines= lines[FLine:LLine]
for x in lines:
    inds.append(int(x.split()[0]))
    y_m_t.append(int(x.split()[1]))
f.close()
y_m_t = np.array(y_m_t)
inds = np.array(inds)
inds=inds -1

#frame numbers of each cluster store in shapshots
#the frames of each cluster may be different
snapshots = []
unique = sorted(np.unique(y_m_t))
for i in range(len(np.unique(y_m_t))):
    snapshots.append(inds[y_m_t == i])

ml_samples = snapshots 

dyn = [150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 92, 162, 149, 161, 131, 137, 157, 158]
ts = [10, 10, 10, 10, 10, 10, 10, 60, 60, 60, 60, 600, 200, 200, 200, 600, 600, 200, 200]
##Edit for your trajectories with length of each trajectory in [dyn] and times step per dcd in [ts]

cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []
cluster6 = []
cluster7 = []
cluster8 = []
cluster9 = []
##Edit the location of your combined now h5 trajectories so they can be loaded in the for loop below 
for row in ml_samples[0]:
  cluster1.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[1]:
  cluster2.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[2]:
  cluster3.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[3]:
  cluster4.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[4]:
  cluster5.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[5]:
  cluster6.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[6]:
  cluster7.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[7]:
  cluster8.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')
for row in ml_samples[8]:
  cluster9.append('/content/gdrive/MyDrive/ensemble/comb/'+str(row)+'.h5')

##Edit there are 9 simualtions in the clustering done hence the range(1, 10)
for c in range(1,10):
  listt = "cluster"+str(c)
  cluster = md.load(eval(listt))
  avg = cluster.xyz.mean(axis=0)
  cluster.xyz = avg
  ff = "/content/gdrive/MyDrive/ensemble/final2/avg{0}.h5".format(c)
  f = md.formats.HDF5TrajectoryFile(ff, 'w')
  f.write(cluster.xyz)
  f.close()
  cluster = md.load(eval(listt))
  tmp = md.load('/content/gdrive/MyDrive/ensemble/final2/avg'+str(c)+'.h5')
  rm = md.rmsd(cluster, tmp, 0)
  indices = rm.argsort()[:50]
  np.savetxt("/content/gdrive/MyDrive/ensemble/out/clusterFrames{0}.csv".format(c),indices, delimiter =" ",  fmt ='% s')
  name='/content/gdrive/MyDrive/ensemble/final/cluster_'+str(c)+'.dcd'
  namepdb = '/content/gdrive/MyDrive/ensemble/final/cluster_'+str(c)+'.pdb'
  cluster[indices].save_dcd(name)
  cluster[indices[1]].save_pdb(namepdb)
##Edit you have to change the location of the files but the code should work on its own 

/usr/local/lib/python3.7/dist-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')
